<a href="https://colab.research.google.com/github/koushikac/ML_Algorithm_projects/blob/main/Knn_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Importing related Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import csv
import warnings
warnings.filterwarnings("ignore")

1. Getting familier with the data

In [15]:
df = pd.read_csv('/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.shape

(891, 12)

In [9]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [10]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Dropping the unwanted columns. Name, Ticket, Fare, Cabin seems unwanted columns, so let's remove them.

In [16]:
df = df.drop('Name', axis=1,)
df = df.drop('Ticket', axis=1,)
#df = df.drop('Fare', axis=1,)
df = df.drop('Cabin', axis=1,)

In [17]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


Combining # siblings and # parents

In [18]:
df['Family'] = df['SibSp'] + df['Parch'] + 1

In [19]:
df = df.drop('SibSp', axis=1,)
df = df.drop('Parch', axis=1,)

In [20]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Family
0,1,0,3,male,22.0,7.2500,S,2
1,2,1,1,female,38.0,71.2833,C,2
2,3,1,3,female,26.0,7.9250,S,1
3,4,1,1,female,35.0,53.1000,S,2
4,5,0,3,male,35.0,8.0500,S,1


In [21]:
df.describe()

,PassengerId,Survived,Pclass,Age,Fare,Family
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,32.204208,1.904602
std,257.353842,0.486592,0.836071,14.526497,49.693429,1.613459
min,1.000000,0.000000,1.000000,0.420000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,20.125000,7.910400,1.000000
50%,446.000000,0.000000,3.000000,28.000000,14.454200,1.000000
75%,668.500000,1.000000,3.000000,38.000000,31.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,512.329200,11.000000


In [22]:
df.Family.sum()

1697

In [23]:
iqr=df.Age.quantile(0.75)-df.Age.quantile(0.25)
upper_threshold=df.Age.quantile(0.75)+(1.5*iqr)
lower_threshold=df.Age.quantile(0.25)-(1.5*iqr)
print(upper_threshold)
print(lower_threshold)

64.8125
-6.6875


# 2. Cleaning the data

In [24]:
df['Age'].isnull().sum()

177

In [32]:
df["Age"] = df["Age"].fillna(df["Age"].mean())

In [33]:
df.describe()

,PassengerId,Survived,Pclass,Age,Fare,Family
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.361582,32.204208,1.904602
std,257.353842,0.486592,0.836071,13.019697,49.693429,1.613459
min,1.000000,0.000000,1.000000,0.420000,0.000000,1.000000
25%,223.500000,0.000000,2.000000,22.000000,7.910400,1.000000
50%,446.000000,0.000000,3.000000,28.000000,14.454200,1.000000
75%,668.500000,1.000000,3.000000,35.000000,31.000000,2.000000
max,891.000000,1.000000,3.000000,80.000000,512.329200,11.000000


In [34]:
df['Age'].isnull().sum()

0

In [29]:
df['Age'].median()

28.0

In [30]:
df['Age'].mean()

29.36158249158249

In [35]:
df.sample(5)

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Family
650,651,0,3,male,28.0,7.8958,S,1
744,745,1,3,male,31.0,7.9250,S,1
669,670,1,1,female,28.0,52.0000,S,2
77,78,0,3,male,28.0,8.0500,S,1
833,834,0,3,male,23.0,7.8542,S,1


In [36]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [40]:
#finding NAN's in Embarked column
df['Embarked'].isna().sum()

0

In [56]:
#Replacing the NAN's with most frequently used one i.e mode(metric that gives us most frequently used value)
print(df["Embarked"].mode())
df["Embarked"] = df["Embarked"].fillna("S")

0    1
dtype: int64


In [57]:
df['Embarked'].describe()

count    891.000000
mean       1.361392
std        0.635673
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        3.000000
Name: Embarked, dtype: float64

In [43]:
# Replacing the categorical value Embarked into numerical value
df["Embarked"].unique()

array(['S', 'C', 'Q'], dtype=object)

**encoding**

In [44]:
df.Embarked.replace(['S', 'C', 'Q'], [1, 2, 3], inplace=True)

In [45]:
df.Sex.replace(['male', 'female'], [1,0], inplace=True)

In [46]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Family
0,1,0,3,1,22.0,7.2500,1,2
1,2,1,1,0,38.0,71.2833,2,2
2,3,1,3,0,26.0,7.9250,1,1
3,4,1,1,0,35.0,53.1000,1,2
4,5,0,3,1,35.0,8.0500,1,1


3. Finding optimal K using cross validation (simple cross validation and KFold cross validation)

In [47]:
# importing libraries
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from collections import Counter
from sklearn.model_selection import cross_validate

In [58]:
X = np.array(df.filter(['Pclass','Sex','Embarked','Family','Age'], axis=1))

In [59]:
y = np.array(df.filter(['Survived'], axis=1))

Splitting the data into train and test data.

In [60]:
from sklearn.model_selection import train_test_split #import the required function
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print('Train set',X_train.shape, y_train.shape)
print('Train set',X_test.shape, y_test.shape)

Train set (712, 5) (712, 1)
Train set (179, 5) (179, 1)


In [61]:
from sklearn.neighbors import KNeighborsClassifier

In [69]:
k=5
model=KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
model

KNeighborsClassifier()

In [70]:
yhat=model.predict(X_test)
yhat

array([1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0])

In [71]:
from sklearn import metrics
print("Train set accuracy :", metrics.accuracy_score(y_train,model.predict(X_train)))
print("Train set accuracy :", metrics.accuracy_score(y_test,yhat))

Train set accuracy : 0.8441011235955056
Train set accuracy : 0.7262569832402235
